In [ ]:
SEARCH_OPTIONS = ["me", "mce"]

import argparse, sys
from Problem import Problem
import copy

from queue import PriorityQueue, Queue
import copy

from PDDLhelp import *

import time
import os

import fnmatch

from collections import Counter

import pandas as pd

import pickle


from pddl.formatter import domain_to_string, problem_to_string
from pddl import parse_domain, parse_problem



In [ ]:
def is_human_action_valid_in_robot_model(action_string, filename):
    with open(filename, 'r') as file:
        for line in file:
            if line.strip() == action_string:
                return True
    return False

In [ ]:
def load_obs(domain_folder_path='', file_name='obs.dat') :
    obs = []
    
    if domain_folder_path == '':
        obs_path = f"{file_name}"
    else:
        obs_path = f"{domain_folder_path}/{file_name}"
        
    
    
    instream = open( obs_path )
    for line in instream :
        line = line.strip()
        obs.append( line )
    instream.close()
    return obs

In [ ]:
def create_problem_templete(input_string, problem_folder):
    
    substring = "(:init"
    everything_before_init = input_string.split(substring)[0]
    
#     print(everything_before_init)
    template = f"{everything_before_init}" \
       "    (:init\n" \
       "%INIT%\n" \
       ")\n" \
       "(:goal\n" \
       "    (and\n" \
       "\t\t%GOAL%\n" \
       ")))"
    
#     print(template)

    
    #File writer
    with open(f"{problem_folder}/problem_template.pddl", "w") as text_file:
        text_file.write(template)
    

In [ ]:
domain_folder_path_list = ['xyz']

In [ ]:
domain_folder_path_list[7:]

In [ ]:
##Create Initial problem template

for domain_folder_path in domain_folder_path_list:
    
    instance_name = domain_folder_path.split('/')[-1]
    print(instance_name)
    print(domain_folder_path)
    
    
    
    init_problem_path = f'{domain_folder_path}/robot_problem.pddl'
    
    init_problem = parse_problem(init_problem_path)
    #write problem templete
    input_problem_as_string = problem_to_string(init_problem)
    create_problem_templete(input_problem_as_string,domain_folder_path)
    

In [ ]:
planner_type = 'optimal'
domain = 'elevator'

column_names = ['planner','domain','instance_name',
                'Real_time_slice_0_for_before_obs',
                'Real_Obs',
                'exp_size',
                'time required for exp'
                ]



for domain_folder_path in domain_folder_path_list:
    
    df_row_lst = []

    # create an empty DataFrame
    df = pd.DataFrame(columns=column_names)
    
    instance_name = domain_folder_path.split('/')[-1]
    print(instance_name)
    print(domain_folder_path)
    
    domain = domain_folder_path.split('/')[-2].split('_')[-1]
    print(domain)
    
    real_obs = load_obs(domain_folder_path=domain_folder_path)
    
    base_folder = domain_folder_path
    
    #Domain file with real PDDL model of robot.
    #Domain file of the problem -m
    model_robot = f"{base_folder}/robot_domain.pddl"


    #Domain file with human model of the robot.
    #Domain file of the problem -n
    model_human = f"{base_folder}/human_domain.pddl"
    
    init_human_problem = f"{base_folder}/robot_problem.pddl"

    #Problem file for robot. -p
    problem_robot = f"{base_folder}/robot_problem.pddl"

    hproblem = f"{base_folder}/robot_problem.pddl"

    #Domain file template for the problem. -t
    domain_template = f"{base_folder}/domain_template.pddl"

    #Problem file template. -r
    problem_template = f"{base_folder}/problem_template.pddl"





    ground = None
    approx =None
    heuristic = None
    # hproblem = None
    plan_file = None
    h_obs = None


    pr_obj = Problem(model_robot, model_human, problem_robot, domain_template,
     ground, approx, heuristic,
     problem_template,h_obs, hproblem, plan_file)




    start_time = time.time()

#     number_of_nodes_expanded = pr_obj.MeSearch()

    end_time = time.time()

    total_time = end_time - start_time

    print("Execution time:", total_time)

#     solution_list_of_sets = [s.difference(pr_obj.human_state) for s in pr_obj.get_solution()]
    solution_list_of_sets = [[1,2,3,4,5]]
    init_exp_len = len(solution_list_of_sets[0])
    print('init_exp_len:', init_exp_len)
    



                # append new data to dataframe
    data_to_append = [planner_type,domain,
        instance_name,
        0, #before obs
        'this is before obs', #extra info to store the real obs/act
        init_exp_len, 
        total_time
    ]   
    
    df_row_lst.append(data_to_append)
    
    # create extension
    df_extended = pd.DataFrame(df_row_lst, columns=column_names)
    # concatenate to original
    out_df = pd.concat([df, df_extended])
    out_df.to_csv(f't_expl_output_{domain}_{instance_name}_{planner_type}.csv', index=False) 


    
    
    #until here everything happened before executing any observation
    #j starts from 0, and this is the real time_slice, so we use j+1 to record the first obs
    for j in range(0, len(real_obs) ) :
        
        
        #observer.obs is a list of obs until time j+1
        obs_until_this_time = real_obs[0:j+1]
        

        
        with open('temp_obs.dat', 'w') as file:
            file.write('\n'.join(obs_until_this_time))  # Write all elements except the last one        
        
        
        #Step 1: Get possible human actions at this obs step
        if j==0:
            #dont pass domain name for the domains without mixed types    
            os.system(f"python t_get_next_possible_actions_list.py -d {model_robot} -i {problem_robot}")
        else:
            os.system(f"python t_get_next_possible_actions_list.py -d {model_robot} -i temp_iterated_problem_robot_at_{j}.pddl")
        
        robot_valid_poss_act_at_this_step_list = f'{instance_name}_robot_possible_actions_at_{j}.dat'        
        
        os.rename('temp_next_possible_actions.dat',robot_valid_poss_act_at_this_step_list)

        human_action_at_this_step  = real_obs[j]
        
        print('Step and human action', j+1,human_action_at_this_step)
        
        
        #Step 2: Check if the observed human action is valid in robot model
        is_valid_act_in_human_model = is_human_action_valid_in_robot_model(human_action_at_this_step, robot_valid_poss_act_at_this_step_list)
        print('is_valid_act_in_human_model:', is_valid_act_in_human_model)
        
        flag_is_obs_valid_in_robot ='not yet'
        
        
        #if obs action is valid in robot model
        #is_valid_act_in_human_model
        if is_valid_act_in_human_model:
            temp_obs = 'temp_obs.dat'
            
            print('temp_obs:', temp_obs)

            #This creates a problem file called temp_iterated_problem.pddl
            #this is how we can iterate the initial state based on the obs sequence
            
            

            
            
            #1: Update robot problem file based on the observation
            os.system(f"python t_upd_prob_file_with_obs_list_not_merged.py -d {model_robot} -i {problem_robot} -o {temp_obs}")        
            
            
            
            
            
            #this is j+1 bc we'll use this for the next step, current problem has been used already
            os.rename('temp_iterated_problem.pddl',f'temp_iterated_problem_robot_at_{j+1}.pddl')
            
#             os.system(f"python t_upd_prob_file_with_obs_list_not_merged.py -d {model_human} -i {problem_robot} -o {temp_obs}")        
#             #this is j+1 bc we'll use this for the next step, current problem has been used already
#             os.rename('temp_iterated_problem.pddl',f'temp_iterated_problem_human_at_{j+1}.pddl')
            
            
            #we are checking the explanations after an obs executed
            #before obs executed, we have already calculated
            
            #this is after obs executed
            print('This is j: ', j)
            

            problem_robot =f'temp_iterated_problem_robot_at_{j+1}.pddl'
            hproblem =f'temp_iterated_problem_human_at_{j+1}.pddl'



            pr_obj_for_obs = Problem(model_robot, model_human, problem_robot, domain_template,
             ground, approx, heuristic,
             problem_template,human_obs=temp_obs, humanProblemFile=init_human_problem, robotPlanFile=None)




            start_time = time.time()

            number_of_nodes_expanded = pr_obj_for_obs.MeSearch()

            end_time = time.time()

            total_time = end_time - start_time

            print("Execution time:", total_time)

            sol_list_of_sets_for_obs_seq = [s.difference(pr_obj_for_obs.human_state) for s in pr_obj_for_obs.get_solution()]

            exp_len_for_obs_seq = len(sol_list_of_sets_for_obs_seq[0])
            print('exp_len_for_obs_seq: ', exp_len_for_obs_seq)

            data_to_append = [planner_type,domain,
                instance_name,
                j+1, #before obs
                real_obs[j], #extra info to store the real obs/act
                exp_len_for_obs_seq, 
                total_time]  

            df_row_lst.append(data_to_append)

            # create extension
            df_extended = pd.DataFrame(df_row_lst, columns=column_names)
            # concatenate to original
            out_df = pd.concat([df, df_extended])
            out_df.to_csv(f't_expl_output_{domain}_{instance_name}_{planner_type}.csv', index=False) 

            
    
                
                
                
                
        #if obs action is not valid in robot's model
        else:
            flag_is_obs_valid_in_robot = 'No'
            reason= f"Obs no :, {j+1}, is not valid in robot model"

                
            data_to_append = [planner_type,domain,
            instance_name,
            j+1, #before obs
            real_obs[j], #extra info to store the real obs/act
            'Not valid in robot model', 
            'N/A'
                ]  
        


            df_row_lst.append(data_to_append)

            # create extension
            df_extended = pd.DataFrame(df_row_lst, columns=column_names)
            # concatenate to original
            out_df = pd.concat([df, df_extended])
            out_df.to_csv(f't_expl_output_{instance_name}.csv', index=False)  # Specify the desired filename 
            
            print('Obs no :', j+1, ' is not valid in robot model')
            
            #don't run the rest of the real obs
            break
    

In [ ]:
pr_obj_for_obs.robotPlanFile

In [ ]:
sol_list_of_sets_for_obs_seq

barman missing= 22-25